In [20]:
#Credits and References
#This code using CityBike API: http://api.citybik.es/v2/

In [21]:
import pandas as pd
import numpy as np
from count_api import CountAPI
import requests

In [22]:
#Authenticate Count
token = "PASTE_TOKEN_HERE"
count = CountAPI()
count.set_api_token(token)

# Connect to City Bike API and see what cities are available

In [23]:
url = "http://api.citybik.es/v2/networks"
resp = requests.get(url)
data = resp.json()

In [24]:
#Turn json response into data frame
companies = []
ids = []
cities = []
country = []
names = []
for i in range(len(data['networks'])):
    companies.append(data['networks'][i]['company'])
    ids.append(data['networks'][i]['id'])
    names.append(data['networks'][i]['name'])
    cities.append(data['networks'][i]['location']['city'])
    country.append(data['networks'][i]['location']['country'])
df = pd.DataFrame({'name':names,'city':cities,'country':country,'company':companies,'id':ids})

In [25]:
all_bikes = count.upload(data = df.to_csv(),name = 'All CityBikes')
visual = all_bikes.upload_visual(x = all_bikes['country']
                                ,y = all_bikes[0]
                                ,aggregates = {'y':'distinct'}
                                ,chart_options = {'type':'bar'})
visual.embed()

## London Bike Share Analysis

In [27]:
london_df = pd.DataFrame()
for bike_co in df[df['city']=='London']['id'][::-1]:
    url = "http://api.citybik.es/v2/networks/"+bike_co
    resp = requests.get(url)
    data = resp.json()
    temp_df = pd.DataFrame.from_dict(data['network']['stations'])
    install_dates = []
    for i in range(len(data['network']['stations'])):
        try:
            install_dates.append(data['network']['stations'][i]['extra']['installDate'])
        except:
            install_dates.append(pd.NaT)
    temp_df['install_dates']= install_dates
    temp_df['bike_co']= bike_co
    london_df = london_df.append(temp_df,ignore_index = True, sort = False)

In [29]:
london_df.head()

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp,install_dates,bike_co
0,1,"{'installDate': '', 'installed': True, 'locked...",22,6900a8774d4b47db2cd8c16b3488d4f2,51.541596,-0.125441,"300235 - York Way, Kings Cross",2018-11-18T15:19:34.135000Z,,santander-cycles
1,22,"{'installDate': '', 'installed': True, 'locked...",3,fc0a260fc584c45809316ef2fbcd9ce7,51.516785,-0.104948,"300220 - Farringdon Street, Holborn",2018-11-18T15:19:34.130000Z,,santander-cycles
2,10,"{'installDate': '1278947280000', 'installed': ...",9,7f3020118e56165ed8b2f61899edb971,51.529163,-0.109971,"001023 - River Street , Clerkenwell",2018-11-18T15:19:27.681000Z,1278947280000,santander-cycles
3,7,"{'installDate': '1278585780000', 'installed': ...",30,67e6c16bce05410ba4b1f0f5000726ea,51.499607,-0.197574,"001018 - Phillimore Gardens, Kensington",2018-11-18T15:19:27.704000Z,1278585780000,santander-cycles
4,29,"{'installDate': '1278240360000', 'installed': ...",3,2d1fd6bf5426f3dc34fae06f43f76f69,51.521284,-0.084606,"001012 - Christopher Street, Liverpool Street",2018-11-18T15:19:27.709000Z,1278240360000,santander-cycles


In [31]:
london_df['install_dates']= pd.to_datetime(london_df['install_dates'],unit = 'ms')
london_df['install_month']= pd.to_datetime(london_df['install_dates']).dt.month
london_df['install_year']= pd.to_datetime(london_df['install_dates']).dt.year

In [35]:
london_df = london_df.drop(['extra'],axis = 1)

In [36]:
#The "name" field contains info about the neighbourhood; this could be useful
def get_borough(name):
    borough = name.split(",")[-1]
    return borough

In [37]:
london_df['borough']=london_df['name'].apply(get_borough)

In [38]:
#Let's find which neighbourhood has the highest city bike usage rates
london_df['usage_per']= 100*(london_df['empty_slots']/(london_df[['empty_slots','free_bikes']].sum(axis = 1)))

In [41]:
new_cols = []
for col in london_df.columns:
    if "_" in col:
        col = col.replace("_",' ')
    else:
        col = col
    new_cols.append(col)

In [42]:
london_df.columns = new_cols

In [45]:
#Generate a new table: 
stations_table = count.upload(data = london_df.to_csv(),name = "London Bikes")


In [46]:
#Show map of London, size is larger for stations w/ more available bikes. Each station is colored by its borough (optional to change coloring to installation year)
visual = stations_table.upload_visual(x = stations_table['longitude'],
                                      y = stations_table['latitude']
                                      ,size = stations_table['free bikes']
                                      #,color = stations_table['install year']
                                      ,color = stations_table['borough']
                                      ,chart_options = {'type':'circle'})
visual.embed()

In [52]:
#What are the boroughs w/ average usage rates over 85%
visual = stations_table.upload_visual(x = stations_table['borough'], 
                                      y = stations_table['usage per']
                                      ,aggregates = {'y':'avg'}
                                      ,chart_options = {'type':'bar'},
                                     filters = [stations_table['usage per'].filter(">=",85)])
visual.embed()

In [47]:
#What years were most of the stations installed?
stations_visual = stations_table.upload_visual(x = stations_table['install year']
                                               , y =stations_table[0]
                                               ,aggregates = {'y':'distinct'}
                                               ,chart_options = {'type':'bar'})

stations_visual.embed()